In [1]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryOutputParser
from pydantic import BaseModel, Field, field_validator
from typing import List
 

In [4]:
"""
在某些情况下，只查看输出就可以修复任何解析错误，但在其他情况下则不行。例如，当输出不仅格式不正确，而且部分不完整时，就是这种情况。
"""
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")
 
parser = PydanticOutputParser(pydantic_object=Action)
 

In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
 

In [6]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")
 

In [7]:
bad_response = '{"action": "search"}'
 

In [8]:
parser.parse(bad_response)
 

ValidationError: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [ ]:
""" output
ValidationError: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
"""

In [9]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
 

In [10]:
fix_parser.parse(bad_response)
 

ValidationError: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [ ]:
""" output
ValidationError: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
"""

In [11]:
from langchain.output_parsers import RetryWithErrorOutputParser
 

In [12]:
retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=llm)
 

In [13]:
retry_parser.parse_with_prompt(bad_response, prompt_value)
 

ValidationError: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [ ]:
""" expected output
Action(action='search', action_input='leo di caprio girlfriend')
"""